import numpy as np
import pandas as pd
import datatable
import itertools as it
from owlready2 import *
import math
import joblib

import pickle

#para depurar los que letras que no están en el codigo ascii
import unicodedata
import functools
import spacy
import stanfordnlp
#from spacy_stanfordnlp import StanfordNLPLanguage
from sklearn.decomposition import LatentDirichletAllocation
from sklearn import metrics
#from wiki_dump_reader import Cleaner, iterate
from sklearn.decomposition import TruncatedSVD

from sklearn.preprocessing import normalize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import pairwise_distances

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import chi2
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize,scale

#from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
#from gensim.models.wrappers import LdaMallet
#from gensim.corpora import Dictionary

#import pyLDAvis
#import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import SVG

import random
from sklearn.externals import joblib


#Operaciones con gráficos
import scattertext as st
from matplotlib import pyplot as plt
from matplotlib.ticker import MaxNLocator

import networkx as nx

import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool, LabelSet, ColumnDataSource, Range1d
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE

#import gensim
#import gensim.corpora as corpora
#from gensim.models import CoherenceModel
#from gensim.models import HdpModel

warnings.filterwarnings("ignore")

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from multiprocessing import  Pool
import math
import scipy.sparse as sp

import vectorizer
importlib.reload(vectorizer)
from vectorizer import  WeightedTfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import *
from sklearn.metrics import classification_report, confusion_matrix



# Reorganizamos el corpus

In [1]:
import joblib
corpus = joblib.load('../bbc_objects/bbc_processed_final')


In [2]:
corpus

,category,text,coref_text,cleaned,entidades,new_target
0,tech,tv future in the hands of viewers with home th...,tv future in the hands of viewers with home th...,tv future hand viewer home theatre system plas...,"{today, five years time, nine months to a yea...",3
1,business,worldcom boss left books alone former worldc...,worldcom boss left books alone former worldc...,worldcom boss leave book worldcom boss bernie ...,"{about $180bn, monday, last two months, 160bn,...",1
2,sport,tigers wary of farrell gamble leicester say ...,tigers wary of farrell gamble leicester say ...,tiger wary farrell gamble leicester tiger wary...,"{another three months, five weeks ago}",5
3,sport,yeading face newcastle in fa cup premiership s...,yeading face newcastle in fa cup premiership s...,yeade face newcastle fa cup premiership newcas...,"{the weekend, sunday, weekend, saturday, third...",2
4,entertainment,ocean s twelve raids box office ocean s twelve...,ocean s twelve raids box office ocean s twelve...,ocean s raid box office ocean s crime caper se...,"{fourth, 21, second, eleven, $110m, $16.1m, op...",0
...,...,...,...,...,...,...
2220,business,cars pull down us retail figures us retail sal...,cars pull down us retail figures us retail sal...,car pull retail figure retail sale fall 0.3 ja...,"{2004, 0.9%, monthly, 0.5%, last august, 0.1%,...",1
2221,politics,kilroy unveils immigration policy ex-chatshow ...,kilroy unveils immigration policy ex-chatshow ...,kilroy unveil immigration policy ex chatshow h...,"{14 000, 2bn}",6
2222,entertainment,rem announce new glasgow concert us band rem h...,rem announce new glasgow concert us band rem h...,rem announce new glasgow concert band rem anno...,"{tuesday 14, 1, 29, 10 000 scottish, 21, four...",4
2223,politics,how political squabbles snowball it s become c...,how political squabbles snowball it s become c...,political squabble snowball s commonplace argu...,"{the last couple of decades, 283 000, one, two...",6


In [4]:
corpusnormalizado = corpus.iloc[:,[3,5]]
corpusnormalizado.head()

,cleaned,new_target
0,tv future hand viewer home theatre system plas...,3
1,worldcom boss leave book worldcom boss bernie ...,1
2,tiger wary farrell gamble leicester tiger wary...,5
3,yeade face newcastle fa cup premiership newcas...,2
4,ocean s raid box office ocean s crime caper se...,0


In [5]:
entidades=joblib.load('./bbc_objects/entidades_bbc')
topics =joblib.load('./bbc_objects/new_bbc_topics_7')

In [6]:
vocab = set ()

for topic in topics:
    for t in topic[1]:
        vocab.add(t[0])

In [7]:
vocab

{'academy',
 'access',
 'accord',
 'account',
 'accuse',
 'actor',
 'actress',
 'add',
 'advantage',
 'agree',
 'airline',
 'alan',
 'album',
 'alex',
 'allow',
 'analyst',
 'andy',
 'anelka',
 'angele',
 'apple',
 'argue',
 'argument',
 'arm',
 'arnold',
 'arsenal',
 'arsene',
 'art',
 'asylum',
 'attack',
 'attend',
 'australia',
 'australian',
 'aviator',
 'award',
 'baby',
 'bafta',
 'balco',
 'ball',
 'ballet',
 'band',
 'bank',
 'barcelona',
 'base',
 'bayern',
 'bbc',
 'beat',
 'believe',
 'bell',
 'benitez',
 'berger',
 'berlin',
 'big',
 'bit',
 'blair',
 'blog',
 'blunkett',
 'book',
 'boost',
 'boro',
 'boss',
 'box',
 'brazilian',
 'break',
 'brentford',
 'britain',
 'british',
 'brown',
 'budget',
 'bush',
 'business',
 'buy',
 'camera',
 'campaign',
 'campbell',
 'captain',
 'car',
 'career',
 'case',
 'cate',
 'category',
 'celtic',
 'centre',
 'ceremony',
 'champion',
 'championship',
 'chance',
 'chancellor',
 'change',
 'charge',
 'charle',
 'charlton',
 'chart',
 'ch

In [8]:
from vectorizer import *
wtfidf = WeightedTfidfVectorizer(vocabulary = vocab) #aquí ya hemos hecho una selección inicial de variables

In [9]:
newdtm = wtfidf.fit_transform(corpusnormalizado)

In [10]:
newdtm

<2225x564 sparse matrix of type '<class 'numpy.float32'>'
	with 94413 stored elements in Compressed Sparse Row format>

In [11]:
joblib.dump(vocab,'./bbc_objects/bbc_vocab_dtm')
joblib.dump(newdtm,'./bbc_objects/bbc_dtm')

['./bbc_objects/bbc_dtm']